In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qadatasetgrp7/aiml-qa-dataset/aiml-qa-dev.xlsx
/kaggle/input/qadatasetgrp7/aiml-qa-dataset/aiml-qa-train.xlsx
/kaggle/input/qadatasetgrp7/aiml-qa-dataset/info.txt
/kaggle/input/qadatasetgrp7/aiml-qa-dataset/aiml-qa-test.xlsx


In [2]:
import os
import numpy as np

In [3]:
eval_file_path ='/kaggle/input/qadatasetgrp7/aiml-qa-dataset/aiml-qa-dev.xlsx'

In [4]:
train_file_path ="/kaggle/input/qadatasetgrp7/aiml-qa-dataset/aiml-qa-train.xlsx"

In [5]:
import pandas as pd

In [6]:
eval_data = pd.read_excel(eval_file_path)
train_data = pd.read_excel(train_file_path)

In [7]:
with open('AIMLDev.txt', 'w') as file:
    for index, row in eval_data.iterrows():
        file.write("Question : " + f"{row['question']}" + " Answer1 : " + f"{row['answer1']}" + " Answer2 : " + f"{row['answer2']}\n")
file.close()

In [8]:
with open('AIMLTrain.txt', 'w') as file:
    for index, row in train_data.iterrows():
        file.write("Question : " + f"{row['question']}" + " Answer : " + f"{row['answer']}\n")
file.close()

In [9]:
!pip uninstall -y transformers accelerate evaluate rouge_score
!pip install transformers accelerate evaluate rouge_score

Found existing installation: transformers 4.33.0
Uninstalling transformers-4.33.0:
  Successfully uninstalled transformers-4.33.0
Found existing installation: accelerate 0.22.0
Uninstalling accelerate-0.22.0:
  Successfully uninstalled accelerate-0.22.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f4efda0bd1a17476c21205bf355931796869965a7014fa3120a996b7c58f5ec8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [10]:
!pip install evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 4.6 MB/s eta 0:00:00


In [11]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00


In [12]:
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
#os.mkdir('/kaggle/working/output/')

In [14]:
import torch
import numpy as np
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
)
from bert_score import BERTScorer
import evaluate

class GPT2Trainer:
    def __init__(self, train_path, eval_path, output_dir, model_name='gpt2', overwrite_output_dir=False,
                 per_device_train_batch_size=1, num_train_epochs=10, save_steps=100000):
        self.train_path = train_path
        self.eval_path = eval_path
        self.output_dir = output_dir
        self.model_name = model_name
        self.overwrite_output_dir = overwrite_output_dir
        self.per_device_train_batch_size = per_device_train_batch_size
        self.num_train_epochs = num_train_epochs
        self.save_steps = save_steps

    def load_data_collator(self, tokenizer, mlm=False):
        return DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=mlm)
    
    
    def load_dataset(self, file_path, tokenizer):
        dataset = LineByLineTextDataset(
                tokenizer=tokenizer,
                file_path=file_path,
                block_size=512
        )

        return dataset

    def postprocess_text(self, preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]
        return preds, labels

    def preprocess_logits_for_metrics(self, logits, labels):
        pred_ids = torch.argmax(logits, dim=-1)
        return pred_ids, labels

#     def compute_metrics(self, eval_preds, tokenizer, rouge_metric, sacrebleu_metric, bert_scorer, labels):
    def compute_metrics(self, eval_preds):
        tokenizer = GPT2Tokenizer.from_pretrained(self.model_name)
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        logits, labels = eval_preds
        preds = logits[0]
        preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        preds, labels = self.postprocess_text(decoded_preds, decoded_labels)

        sacrebleu_metric = evaluate.load("sacrebleu")
        rouge_metric = evaluate.load('rouge')
        bertscore = evaluate.load("bertscore")
        rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
        sacrebleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels, lowercase=True)
        bert_result = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")

        return {
        "R1": round(rouge_result["rouge1"], 4),
        "R2": round(rouge_result["rouge2"], 4),
        "RL": round(rouge_result["rougeL"], 4),
        "RLsum": round(rouge_result["rougeLsum"], 4),
        "bleu": round(sacrebleu_result["score"], 4),
        "precision1":round(bert_result["precision"][0], 4),
        "precision2":round(bert_result["precision"][1], 4),
        "recall1":round(bert_result["recall"][0], 4),
        "recall2":round(bert_result["recall"][1], 4),
        "f1-score1":round(bert_result["f1"][0], 4),
        "f1-score2":round(bert_result["f1"][1], 4)
    }
    def train_model(self):
        tokenizer = GPT2Tokenizer.from_pretrained(self.model_name)
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

        model = GPT2LMHeadModel.from_pretrained(self.model_name)
        model.resize_token_embeddings(len(tokenizer))
        model.save_pretrained(self.output_dir)

        train_dataset = self.load_dataset(self.train_path, tokenizer)
        eval_dataset = self.load_dataset(self.eval_path, tokenizer)

        data_collator = self.load_data_collator(tokenizer)

        tokenizer.save_pretrained(self.output_dir)

        training_args = TrainingArguments(
            output_dir=self.output_dir,
            evaluation_strategy="epoch",
            learning_rate=1e-5,
            overwrite_output_dir=self.overwrite_output_dir,
            per_device_train_batch_size=self.per_device_train_batch_size,
            per_device_eval_batch_size=1,
            num_train_epochs=self.num_train_epochs
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            preprocess_logits_for_metrics=self.preprocess_logits_for_metrics,
            compute_metrics=self.compute_metrics
        )

        trainer.train()
        trainer.save_model()

if __name__ == "__main__":
    train_path = "/kaggle/working/AIMLTrain.txt"
    eval_path = "/kaggle/working/AIMLDev.txt"
    output_dir = "/kaggle/output"

    bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)
    
    trainer = GPT2Trainer(train_path, eval_path, output_dir)
    trainer.train_model()


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50258. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser her

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,R1,R2,Rl,Rlsum,Bleu,Precision1,Precision2,Recall1,Recall2,F1-score1,F1-score2
1,4.933200,4.904433,0.496800,0.188500,0.372700,0.373100,14.848500,0.884400,0.855000,0.920700,0.868000,0.902200,0.861400
2,4.058900,3.679157,0.508500,0.198600,0.385900,0.386400,16.196200,0.886100,0.878000,0.924300,0.887600,0.904800,0.882800
3,3.731600,3.464115,0.511800,0.202300,0.392200,0.392600,16.603500,0.882400,0.879000,0.921200,0.889100,0.901400,0.884000
4,3.563600,3.445850,0.509000,0.204600,0.394000,0.394400,16.863700,0.886800,0.878500,0.922400,0.889800,0.904300,0.884100
5,3.579600,3.328812,0.509900,0.206700,0.394500,0.394900,17.251100,0.887800,0.878500,0.922800,0.889800,0.905000,0.884100
6,3.474600,3.299269,0.510200,0.208900,0.396800,0.397000,17.619200,0.887800,0.880300,0.922800,0.890500,0.905000,0.885300
7,3.475600,3.236306,0.511500,0.209600,0.396900,0.397400,17.661100,0.887800,0.881400,0.922800,0.889500,0.905000,0.885500
8,3.299600,3.228451,0.512300,0.210000,0.398400,0.398800,17.862500,0.889600,0.881400,0.923600,0.889500,0.906200,0.885500
9,3.342300,3.204564,0.513300,0.210700,0.400000,0.400100,17.936600,0.889600,0.881400,0.923600,0.889500,0.906200,0.885500
10,3.379800,3.203324,0.513600,0.210500,0.400300,0.400400,17.941900,0.889600,0.881400,0.923600,0.889500,0.906200,0.885500


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro